In [ ]:
!unzip "furniture_model_dataset.zip"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Parameter
img_height, img_width = 224, 224
batch_size = 32

# Train data generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Validation
val_datagen = ImageDataGenerator(rescale=1./255)

# Load training data
train_generator = train_datagen.flow_from_directory(
    '/content/train',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load validation data
validation_generator = val_datagen.flow_from_directory(
    '/content/validation',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 88 images belonging to 14 classes.
Found 88 images belonging to 14 classes.


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout

# Load VGG16 + higher level layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of VGG16
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(14, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [ ]:
# Train model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=25
)

Epoch 1/25
2/2 [==============================] - 75s 54s/step - loss: 5.4585 - accuracy: 0.0893 - val_loss: 5.6552 - val_accuracy: 0.2188
Epoch 2/25
2/2 [==============================] - 73s 54s/step - loss: 7.7071 - accuracy: 0.1071 - val_loss: 3.8800 - val_accuracy: 0.3750
Epoch 3/25
2/2 [==============================] - 72s 58s/step - loss: 6.3029 - accuracy: 0.1786 - val_loss: 3.6775 - val_accuracy: 0.2031
Epoch 4/25
2/2 [==============================] - 72s 58s/step - loss: 5.0324 - accuracy: 0.1429 - val_loss: 1.4015 - val_accuracy: 0.6250
Epoch 5/25
2/2 [==============================] - 72s 58s/step - loss: 2.9646 - accuracy: 0.2857 - val_loss: 0.9913 - val_accuracy: 0.7344
Epoch 6/25
2/2 [==============================] - 72s 54s/step - loss: 2.4725 - accuracy: 0.4107 - val_loss: 1.0493 - val_accuracy: 0.6719
Epoch 7/25
2/2 [==============================] - 77s 59s/step - loss: 2.3725 - accuracy: 0.3281 - val_loss: 0.8051 - val_accuracy: 0.7500
Epoch 8/25
2/2 [===========

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Dictionary untuk mapping kelas ke nama kursi
class_indices = train_generator.class_indices
index_to_class = {v: k for k, v in class_indices.items()}

def predict_image(img_path):
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = model.predict(img_array)
    class_index = np.argmax(prediction)
    class_name = index_to_class[class_index]
    one_hot_encoded = np.zeros(14)
    one_hot_encoded[class_index] = 1

    return one_hot_encoded, class_name

# Penggunaan fungsi predict_image
img_path = '/content/train/kursi01/kursi01.jpg'
one_hot, class_name = predict_image(img_path)
print(f"Predicted one-hot: {one_hot}, Class name: {class_name}")


1/1 [==============================] - 1s 688ms/step
Predicted one-hot: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], Class name: kursi01


In [ ]:
# Menyimpan model ke file model.h5
model.save('model_furniture.h5')

# Menyimpan model dalam format TensorFlow SavedModel
model.save('saved_furniture')
